# Manipulate atomic configurations

This Notebook provides a quick outline on using atomman to manipulate atomic configurations.  

In [1]:
import numpy as np

import atomman as am

## 1. Atomic configuration basics

In atomman, atomic configurations are represented using three basic objects

- __Box__ represents the system box using three lattice vectors a, b, c, and an origin coordinate.  It has conversion tools to different representations as well as utility methods for crystal symmetry, Miller vectors and planes, and identifying if points are inside/outside.
- __Atoms__ represents a list of atoms.  By default, all atoms have an integer atype and a 3-coordinate pos.  Other properties can be freely assigned as long as the value for each atom is of the same data type and shape.
- __System__ combines a Box object, an Atoms object, periodic boundary condition flags, and the list atom model symbols.  Built-in methods support simple atomic analysis, manipulations, and conversion to alternate atomic configuration representations.

In [3]:
# Simple example: manually define an fcc unit cell
box = am.Box.cubic(a=4.05)                  # class methods exist for all 7 crystal symmetry families

atoms = am.Atoms(atype = 1, 
                 pos = [[0.0, 0.0, 0.0],
                        [0.5, 0.5, 0.0],
                        [0.5, 0.0, 0.5], 
                        [0.0, 0.5, 0.5]])

ucell = am.System(atoms = atoms,
                  box = box,
                  scale = True,              # scale=True scales atom pos to box vectors
                  symbols = 'Al')  

print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


## 2. Box tool examples

### 2.1. Box representations

The box parameters can be retrieved in a variety of different formats.  You can also define a box with any complete set of parameters.

In [3]:
print('vects')
print(ucell.box.vects)
print()

print('origin')
print(ucell.box.origin)
print()

print('avect, bvect, cvect')
print(ucell.box.avect)
print(ucell.box.bvect)
print(ucell.box.cvect)
print()

print('a, b, c, alpha, beta, gamma')
print(ucell.box.a, ucell.box.b, ucell.box.c)
print(ucell.box.alpha, ucell.box.beta, ucell.box.gamma)
print()

print('box lengths, lo/hi, and tilts')
print(ucell.box.lx, ucell.box.ly, ucell.box.lz)
print(ucell.box.xlo, ucell.box.xhi, ucell.box.ylo, ucell.box.yhi, ucell.box.zlo, ucell.box.zhi)
print(ucell.box.xy, ucell.box.xz, ucell.box.yz)

vects
[[4.05 0.   0.  ]
 [0.   4.05 0.  ]
 [0.   0.   4.05]]

origin
[0. 0. 0.]

avect, bvect, cvect
[4.05 0.   0.  ]
[0.   4.05 0.  ]
[0.   0.   4.05]

a, b, c, alpha, beta, gamma
4.05 4.05 4.05
90.0 90.0 90.0

box lengths, lo/hi, and tilts
4.05 4.05 4.05
0.0 4.05 0.0 4.05 0.0 4.05
0.0 0.0 0.0


### 2.2. Basic box utilities

In [4]:
print(ucell.box.volume)

66.43012499999999


In [5]:
print(ucell.box.reciprocal_vects)

[[0.24691358 0.         0.        ]
 [0.         0.24691358 0.        ]
 [0.         0.         0.24691358]]


In [9]:
print(ucell.box.inside([[0.0, 0.0, 0.0],
                        [-0.1, 0.0, 0.0]]))

[ True False]


### 2.3. Crystal symmetry methods

There is one "is" method for each of the 7 families

In [10]:
print(ucell.box.iscubic())

True


In [11]:
print(ucell.box.ishexagonal())

False


In [12]:
print(ucell.box.identifyfamily())

cubic


### 2.4. Miller vector and plane operations, and pos conversions

In [13]:
print(ucell.box.vector_crystal_to_cartesian([1,1,1]))

[4.05 4.05 4.05]


In [14]:
print(ucell.box.plane_crystal_to_cartesian([1,1,1]))

[0.57735027 0.57735027 0.57735027]


In [15]:
print(ucell.box.d_hkl([1,1,1]))

2.3382685902179863


In [16]:
print(ucell.box.position_relative_to_cartesian([0.5, 0.5, 0.5]))

[2.025 2.025 2.025]


In [17]:
print(ucell.box.position_cartesian_to_relative([2.025, 2.025, 2.025]))

[0.5 0.5 0.5]


## 3. Atoms tools examples

### 3.1. Basic info

In [18]:
print(ucell.atoms.natoms)

4


In [19]:
print(ucell.atoms.natypes)

1


### 3.2. Getting and setting atomic properties

By default, Atoms have atype (integer atomic type) and pos (3D coordinates).  Any other per-atom properties can be assigned as long as each atom has a value and the value's data type and shape are the same for each atom.

prop() can be used to see names of set properties, set values and retrieve values.

In [20]:
print(ucell.atoms.prop())

['atype', 'pos']


But it is typically easier to interact with properties as attributes

In [21]:
print(ucell.atoms.pos)

[[0.    0.    0.   ]
 [2.025 2.025 0.   ]
 [2.025 0.    2.025]
 [0.    2.025 2.025]]


In [23]:
# Set per-atom 3x3 stress tensors of 0's for all atoms
ucell.atoms.stress = np.zeros([4,3,3])

In [24]:
print(ucell.atoms.prop())

['atype', 'pos', 'stress']


In [25]:
# Show stress tensor for atom 0
print(ucell.atoms.stress[0])

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### 3.3. View as a DataFrame

You can also convert the data to a df to view values in a table format.  **Caution!** The df is built by copying the data into a DataFrame and therefore could cause memory issues if the number of atoms is very large!

In [26]:
ucell.atoms.df()

,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2]
0,1,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2.025,2.025,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2.025,0.000,2.025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.000,2.025,2.025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. System tools examples

### 4.1. Basic info

In [27]:
print(ucell.natoms)

4


In [28]:
print(ucell.composition)

Al


### 4.2. Position analysis

- __dvect()__ finds shortest vectors between points taking account of pbcs. Points can be given either as coordinates or atom ids.
- __dmag()__ finds magnitudes of the shortest vectors between points taking account of pbcs.  Points can be given either as coordinates or atom ids.
- __r0()__ finds the shortest interatomic separation distance in the cell.  Can be costly for large systems.

In [29]:
print(ucell.dvect(0, [1,2,3]))  # Find dvects between first atom and the rest

[[2.025 2.025 0.   ]
 [2.025 0.    2.025]
 [0.    2.025 2.025]]


In [30]:
print(ucell.dmag([2.025, 2.025, 2.025], [0,1,2,3]))  # Find dmags between center of ucell and the atom points

[3.50740289 2.025      2.025      2.025     ]


In [31]:
print(ucell.r0())

2.8637824638055176


### 4.3. System manipulations

- __rotate()__ the system using Miller(-Bravais) integer vectors relative to the current box vectors. Creates a new system such that atoms remain coherent across periodic boundaries.
- __supersize()__ the system by multiplying along the box directions in postive and/or negative directions. 
- __box_set()__ changes the box dimensions with the option to leave atoms at either absolute Cartesian or box-vector-relative positions.
- __atoms_extend()__ creates a new System with the current atoms plus extras.
- __atoms_ix()__ creates a new System based on taking a slice of the old system's atoms list.  Numpy-style indexing is used for selecting the atoms.
- __wrap()__ adjusts the system so all atoms are within the box boundaries. This is done by wrapping atoms around periodic boundaries and extending non-periodic boundaries as needed.

In [32]:
# Rotate such that the [111] direction is along the x-axis
uvws = [
    [ 1, 1, 1],
    [ 1,-2, 1],
    [ 1, 0,-1]
]
rcell = ucell.rotate(uvws)
print(rcell)

avect =  [ 7.015,  0.000,  0.000]
bvect =  [ 0.000,  9.920,  0.000]
cvect =  [ 0.000,  0.000,  5.728]
origin = [ 0.000,  0.000,  0.000]
natoms = 24
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'stress']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   2.480 |   4.296
      1 |       1 |   0.000 |   9.920 |   2.864
      2 |       1 |   0.000 |   7.440 |   4.296
      3 |       1 |   0.000 |   7.440 |   1.432
      4 |       1 |   2.338 |   9.094 |   4.296
      5 |       1 |   0.000 |   2.480 |   1.432
      6 |       1 |   0.000 |   4.960 |   0.000
      7 |       1 |   7.015 |   4.960 |   2.864
      8 |       1 |   2.338 |   4.134 |   4.296
      9 |       1 |   2.338 |   6.614 |   2.864
     10 |       1 |   2.338 |   4.134 |   1.432
     11 |       1 |   4.677 |   5.787 |   4.296
     12 |       1 |   0.000 |   9.920 |   0.000
     13 |       1 |   2.338 |   1.653 |   0.000
     14 |       1 |   2.338

In [33]:
# Multiply system along all three box directions. (0,0,0) coordinate is in the center of the system.
system = rcell.supersize((-4, 4), (-3, 3), (-5, 5))

print(system.box)
print(system.natoms)

avect =  [56.118,  0.000,  0.000]
bvect =  [ 0.000, 59.523,  0.000]
cvect =  [ 0.000,  0.000, 57.276]
origin = [-28.059, -29.761, -28.638]
11520


### 4.4. Neighbor lists

In [34]:
nlist = system.neighborlist(cutoff = 1.1 * ucell.r0())

In [35]:
print(nlist.coord.mean())

12.0


In [36]:
print(nlist[0])

[   5    8   14  175  183  184  191  961 1157 1158 1165 2124]
